# Hedonic Pricing

We often try to predict the price of an asset from its observable characteristics. This is generally called **hedonic pricing**: How do the unit's characteristics determine its market price?

In the lab folder, there are three options: housing prices in pierce_county_house_sales.csv, car prices in cars_hw.csv, and airbnb rental prices in airbnb_hw.csv. If you know of another suitable dataset, please feel free to use that one.

1. Clean the data and perform some EDA and visualization to get to know the data set.
2. Transform your variables --- particularly categorical ones --- for use in your regression analysis.
3. Implement an ~80/~20 train-test split. Put the test data aside.
4. Build some simple linear models that include no transformations or interactions. Fit them, and determine their RMSE and $R^2$ on the both the training and test sets. Which of your models does the best?
5. Include transformations and interactions, and build a more complex model that reflects your ideas about how the features of the asset determine its value. Determine its RMSE and $R^2$ on the training and test sets. How does the more complex model your build compare to the simpler ones?
6. Summarize your results from 1 to 5. Have you learned anything about overfitting and underfitting, or model selection?
7. If you have time, use the sklearn.linear_model.Lasso to regularize your model and select the most predictive features. Which does it select? What are the RMSE and $R^2$? We'll cover the Lasso later in detail in class.



In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import train_test_split


df = pd.read_csv('/content/cars_hw (1).csv')
df.head()

,Unnamed: 0,Make,Make_Year,Color,Body_Type,Mileage_Run,No_of_Owners,Seating_Capacity,Fuel_Type,Transmission,Transmission_Type,Price
0,1,Volkswagen,2017,silver,sedan,44611,1st,5,diesel,7-Speed,Automatic,657000
1,2,Hyundai,2016,red,crossover,20305,1st,5,petrol,5-Speed,Manual,682000
2,3,Honda,2019,white,suv,29540,2nd,5,petrol,5-Speed,Manual,793000
3,4,Renault,2017,bronze,hatchback,35680,1st,5,petrol,5-Speed,Manual,414000
4,5,Hyundai,2017,orange,hatchback,25126,1st,5,petrol,5-Speed,Manual,515000


In [22]:
#cleaning
df['No_of_Owners'] = df['No_of_Owners'].apply(lambda x: x[:-2] if len(x) > 2 else x)
df['No_of_Owners'] = pd.to_numeric(df['No_of_Owners'], errors='coerce')
df.head()

,Unnamed: 0,Make,Make_Year,Color,Body_Type,Mileage_Run,No_of_Owners,Seating_Capacity,Fuel_Type,Transmission,Transmission_Type,Price
0,1,Volkswagen,2017,silver,sedan,44611,1,5,diesel,7-Speed,Automatic,657000
1,2,Hyundai,2016,red,crossover,20305,1,5,petrol,5-Speed,Manual,682000
2,3,Honda,2019,white,suv,29540,2,5,petrol,5-Speed,Manual,793000
3,4,Renault,2017,bronze,hatchback,35680,1,5,petrol,5-Speed,Manual,414000
4,5,Hyundai,2017,orange,hatchback,25126,1,5,petrol,5-Speed,Manual,515000


In [47]:
#3

X = df[['Make', 'Make_Year','Mileage_Run','No_of_Owners', 'Seating_Capacity']]
y = df['Price']

# Perform 80/20 split

X = pd.concat([X[['Make_Year','Mileage_Run','No_of_Owners','Seating_Capacity']],
                     pd.get_dummies(X['Make'], dtype='int')], axis = 1)
#Z_test = pd.concat([X_test[['Make_Year','Mileage_Run','No_of_Owners']],
                    # pd.get_dummies(X_test['Make'], dtype='int')], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [43]:
#4

X = df[['Make_Year','Mileage_Run']]
y = df['Price']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

reg = linear_model.LinearRegression(fit_intercept=False).fit(X_train,y_train) # Run regression


y_hat = reg.predict(X_test)
print('Rsq: ', reg.score(X_test,y_test)) # R2
rmse = np.sqrt( np.mean( (y_test - y_hat)**2 ))
print('RMSE: ', rmse) # R2

results = pd.DataFrame({'variable':reg.feature_names_in_, 'coefficient': reg.coef_}) # Regression coefficients
results

Rsq:  -0.0023964129347520124
RMSE:  375713.25904185267


,variable,coefficient
0,Make_Year,426.90185
1,Mileage_Run,-2.89502


In [46]:
y = df['Price']

X = df[['No_of_Owners','Seating_Capacity']]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

reg = linear_model.LinearRegression(fit_intercept=False).fit(X_train,y_train) # Run regression


y_hat = reg.predict(X_test)
print('Rsq: ', reg.score(X_test,y_test)) # R2
rmse = np.sqrt( np.mean( (y_test - y_hat)**2 ))
print('RMSE: ', rmse) # R2

results = pd.DataFrame({'variable':reg.feature_names_in_, 'coefficient': reg.coef_}) # Regression coefficients
results

Rsq:  0.06006556950000019
RMSE:  358003.6767722667


,variable,coefficient
0,No_of_Owners,-109563.797315
1,Seating_Capacity,170655.966581


My two models ran with very similar outputs. When not being able to use "Make" it was very underfitting. Of the two I did, 'No_of_Owners','Seating_Capacity' did the best, but not by much. The difference in RMSE's was only 375713 versus 358003.

In [48]:
#5

X = df[['Make', 'Make_Year','Mileage_Run','No_of_Owners','Seating_Capacity']]
y = df['Price']

# Perform 80/20 split

X = pd.concat([X[['Make_Year','Mileage_Run','No_of_Owners','Seating_Capacity']],
                     pd.get_dummies(X['Make'], dtype='int')], axis = 1)
#Z_test = pd.concat([X_test[['Make_Year','Mileage_Run','No_of_Owners']],
                    # pd.get_dummies(X_test['Make'], dtype='int')], axis = 1)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

reg = linear_model.LinearRegression(fit_intercept=False).fit(X_train,y_train) # Run regression


y_hat = reg.predict(X_test)
print('Rsq: ', reg.score(X_test,y_test)) # R2
rmse = np.sqrt( np.mean( (y_test - y_hat)**2 ))
print('RMSE: ', rmse) # R2

results = pd.DataFrame({'variable':reg.feature_names_in_, 'coefficient': reg.coef_}) # Regression coefficients
results


Rsq:  0.6338647974160687
RMSE:  199510.58745639262


,variable,coefficient
0,Make_Year,6.494349e+04
1,Mileage_Run,2.031133e+00
2,No_of_Owners,-2.604210e+04
3,Seating_Capacity,5.226139e+04
4,Chevrolet,-1.306893e+08
5,Datsun,-1.309361e+08
6,Ford,-1.306235e+08
7,Honda,-1.304980e+08
8,Hyundai,-1.305889e+08
9,Jeep,-1.298210e+08


5- This model did much better. The RMSE went from 358003 to 199510. While still underfitted, the predicted price is much closer to the actual data.

6- I learned that by adding in variables, it can make the predictions much closer to the real data. By only including a few variables for some of the more simple models, there is a lot of underfitting but as you add on variables, that gap closes. However, I would suspect that if we were to add in more and more variables that it would become overfitted. It is important to find the balance between different variables you are using and what kind of models.